In [1]:
import os
os.getcwd()

'/Users/danielvella/Desktop/deepchem/contrib/one_shot_models/examples'

# Sequential Support Graph

In [22]:
class SequentialSupportGraph(object):
    def __init__(self, n_feat):
        """
        Parameters
        ----------
        n_feat: int
          Number of atomic features.
        """
        self.graph = tf.Graph()
        with self.graph.as_default():
          # Create graph topology and x
            self.test_graph_topology = GraphTopology(n_feat, name='test')
            self.support_graph_topology = GraphTopology(n_feat, name='support')
            self.test = self.test_graph_topology.get_atom_features_placeholder()
            self.support = self.support_graph_topology.get_atom_features_placeholder()

        # Keep track of the layers
        self.layers = []
        # Whether or not we have used the GraphGather layer yet
        self.bool_pre_gather = True

    def add(self, layer):
        """Adds a layer to both test/support stacks.

        Note that the layer transformation is performed independently on the
        test/support tensors.
        """

        with self.graph.as_default():
            self.layers.append(layer)

            # Update new value of x
            if type(layer).__name__ in ['GraphConv', 'GraphGather', 'GraphPool']:
                assert self.bool_pre_gather, "Cannot apply graphical layers after gather."

                self.test = layer([self.test] + self.test_graph_topology.topology)
                self.support = layer([self.support] +
                                 self.support_graph_topology.topology)
            else:
                self.test = layer(self.test)
                self.support = layer(self.support)

            if type(layer).__name__ == 'GraphGather':
                self.bool_pre_gather = False  # Set flag to stop adding topology

    def add_test(self, layer): 
        """Adds a layer to test."""
    
        with self.graph.as_default():
            self.layers.append(layer)

            # Update new value of x
            if type(layer).__name__ in ['GraphConv', 'GraphPool', 'GraphGather']:
                self.test = layer([self.test] + self.test_graph_topology.topology)
            else:
                self.test = layer(self.test)

    def add_support(self, layer):
        """Adds a layer to support."""
        with self.graph.as_default():
            self.layers.append(layer)

            # Update new value of x
            if type(layer).__name__ in ['GraphConv', 'GraphPool', 'GraphGather']:
                self.support = layer([self.support] +
                                 self.support_graph_topology.topology)
            else:
                self.support = layer(self.support)

    def join(self, layer):
        """Joins test and support to a two input two output layer"""
        with self.graph.as_default():
            self.layers.append(layer)
            self.test, self.support = layer([self.test, self.support])

    def get_test_output(self):
        return self.test

    def get_support_output(self):
        return self.support

    def return_outputs(self):
        return [self.test] + [self.support]

    def return_inputs(self):
        return (self.test_graph_topology.get_inputs() +
            self.support_graph_topology.get_inputs())


# MUV Siamese One Fold

In [3]:
%cd ../../..
print(os.getcwd())

/Users/danielvella/Desktop/deepchem
/Users/danielvella/Desktop/deepchem


In [4]:
import numpy as np
import deepchem as dc
import tensorflow as tf

import sys
import os

# sys.path.append("/Users/danielvella/Desktop/deepchem/")

from examples.low_data.datasets import load_muv_convmol
from contrib.one_shot_models.graph_topology import GraphTopology

# %run '/Users/danielvella/Desktop/deepchem/examples/low_data/datasets'
# %run '/Users/danielvella/Desktop/deepchem/contrib/one_shot_models/graph_topology'

In [5]:
print(os.getcwd())

/Users/danielvella/Desktop/deepchem


In [6]:
# Number of folds for split 
K = 4

# num positive/negative ligands
n_pos = 10
n_neg = 10

# Set batch sizes for network
test_batch_size = 128
support_batch_size = n_pos + n_neg
nb_epochs = 1
n_train_trials = 2000
n_eval_trials = 20
n_steps_per_trial = 1
learning_rate = 1e-4
log_every_n_samples = 50

# Number of features on conv-mols
n_feat = 75

In [7]:
muv_tasks, dataset, transformers = load_muv_convmol()

# Define metric
metric = dc.metrics.Metric(dc.metrics.roc_auc_score, mode="classification")

task_splitter = dc.splits.TaskSplitter()
fold_datasets = task_splitter.k_fold_split(dataset, K)

train_folds = fold_datasets[:-1]
train_dataset = dc.splits.merge_fold_datasets(train_folds)
test_dataset = fold_datasets[-1]

smiles_field is deprecated and will be removed in a future version of DeepChem.Use feature_field instead.


About to load MUV dataset.
About to featurize MUV dataset.


/Users/danielvella/Desktop/deepchem/deepchem/data/data_loader.py:162: FutureWarning: featurize() is deprecated and has been renamed to create_dataset().featurize() will be removed in DeepChem 3.0
  "featurize() will be removed in DeepChem 3.0", FutureWarning)


About to transform data
Assigning extra tasks to last fold due to uneven split


In [9]:
tf.__version__

'2.2.0'

In [23]:
# Train support model on train
support_model = SequentialSupportGraph(n_feat)

# Add layers
support_model.add(dc.models.layers.GraphConv(64, n_feat, activation_fn='relu'))
support_model.add(dc.nn.GraphPool())
support_model.add(dc.nn.GraphConv(128, 64, activation='relu'))
support_model.add(dc.nn.GraphPool())
support_model.add(dc.nn.GraphConv(64, 128, activation='relu'))
support_model.add(dc.nn.GraphPool())
support_model.add(dc.nn.Dense(128, 64, activation='tanh'))

support_model.add_test(dc.nn.GraphGather(test_batch_size, activation='tanh'))
support_model.add_support(
    dc.nn.GraphGather(support_batch_size, activation='tanh'))

model = dc.models.SupportGraphClassifier(
    support_model,
    test_batch_size=test_batch_size,
    support_batch_size=support_batch_size,
    learning_rate=learning_rate)

model.fit(
    train_dataset,
    nb_epochs=nb_epochs,
    n_episodes_per_epoch=n_train_trials,
    n_pos=n_pos,
    n_neg=n_neg,
    log_every_n_samples=log_every_n_samples)
mean_scores, std_scores = model.evaluate(
    test_dataset, metric, n_pos, n_neg, n_trials=n_eval_trials)

ValueError: in user code:

    /Users/danielvella/Desktop/deepchem/deepchem/models/layers.py:170 call  *
        split_features = tf.split(atom_features, deg_slice[:, 1])
    /opt/miniconda3/envs/deepchem/lib/python3.6/site-packages/tensorflow/python/ops/array_ops.py:1959 split  **
        raise ValueError("Cannot infer num from shape %s" % num_or_size_splits)

    ValueError: Cannot infer num from shape Tensor("graph_conv_6/strided_slice:0", shape=(None, 2), dtype=int32)


In [19]:
print("Mean Scores on evaluation dataset")
print(mean_scores)
print("Standard Deviations on evaluation dataset")
print(std_scores)
print("Median of Mean Scores")
print(np.median(np.array(mean_scores.values())))

Mean Scores on evaluation dataset


NameError: name 'mean_scores' is not defined